# Analiza zadnjih 100 oglasov na portalu avto.net (dne 23.10.2023)

Na portalu [avto.net](https://www.avto.net/) je posebna [stran](https://www.avto.net/Ads/results_100.asp?oglasrubrika=1&prodajalec=2), ki vsebuje zadnjih 100 objavljenih oglasov (stran se sproti spreminja).



In [168]:
#s kodo v tem okvirčku izluščimo html iz spletne podstrani, ki vsebuje zadnjih 100 oglasov

headers = {
    'Accept': 'text/html, image/avif, image/webp, image/apng, image/svg+xml, */ *; q=0.8',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US, en;q=0.9',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-User': '?1',
    'Sec-Fetch-Site': 'none',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'
}

data_request = requests.get('https://www.avto.net/Ads/results_100.asp?', headers=headers)
soup = BeautifulSoup(data_request.content, 'html.parser')
vsebina_html = soup.prettify()
with open("zadnjih_100.html", "w", encoding='utf-8') as f:
     print(vsebina_html, file=f)


# print(data_request.status_code)

In [ ]:
#s kodo v tem okvirčku bomo iz 